<a href="https://colab.research.google.com/github/RAGISHIVANAND/DEEP_LEARNING/blob/main/pytorch_Convolutional_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Importing the torch libraries***

In [6]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
#Device configuration
device = torch.device('cuda:0'if torch.cuda.is_available()else'cpu')

#Hyperparameters
num_epochs=5
num_classes=10
batch_size=100
learning_rate=0.001

In [3]:
#MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='inputdata',
                                           train=True,
                                           transform = transforms.ToTensor(),
                                           download=True)
test_dataset= torchvision.datasets.MNIST(root='inputdata',
                                           train=False,
                                           transform=transforms.ToTensor())
#Dataloader
train_loader=   torch.utils.data.DataLoader(dataset=train_dataset,
                                            batch_size=batch_size,
                                            shuffle=True)
test_loader=torch.utils.data.DataLoader(dataset=test_dataset,
                                        batch_size=batch_size,
                                        shuffle=False)

100%|██████████| 9912422/9912422 [00:00<00:00, 88380921.62it/s]


Extracting inputdata/MNIST/raw/train-images-idx3-ubyte.gz to inputdata/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 33416743.12it/s]


Extracting inputdata/MNIST/raw/train-labels-idx1-ubyte.gz to inputdata/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 26534674.40it/s]


Extracting inputdata/MNIST/raw/t10k-images-idx3-ubyte.gz to inputdata/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 9071680.37it/s]


Extracting inputdata/MNIST/raw/t10k-labels-idx1-ubyte.gz to inputdata/MNIST/raw



In [30]:
#convolutional neural network(two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self,num_classes=10):
        super(ConvNet,self).__init__()
        self.layer1=nn.Sequential(
            nn.Conv2d(1,16,kernel_size=5,stride=1,padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2))
        self.layer2=nn.Sequential(
            nn.Conv2d(16,32,kernel_size=5,stride=1,padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2))
        self.fc=nn.Linear(7*7*32,num_classes)
    def forward(self,x):
        out=self.layer1(x)
        out=self.layer2(out)
        out=out.reshape(out.size(0),-1)
        out=self.fc(out)
        return out    

In [31]:
model = ConvNet(num_classes).to(device)

#Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [32]:
#Train the model
total_step = len(train_loader)
for epoch in range (num_epochs):
    for i,(images,labels) in enumerate(train_loader):
        
        images = images.to(device)
        labels=labels.to(device)

        #Forward pass
        outputs = model(images)
        loss = criterion(outputs,labels)

        #Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if(i+1)% 100 == 0:
           print('Epoch[{}/{}],step[{}/{}],Loss:{:.4f}'
                 .format(epoch+1,num_epochs,i+1,total_step,loss.item()))

Epoch[1/5],step[100/600],Loss:0.1195
Epoch[1/5],step[200/600],Loss:0.2192
Epoch[1/5],step[300/600],Loss:0.0889
Epoch[1/5],step[400/600],Loss:0.0765
Epoch[1/5],step[500/600],Loss:0.0976
Epoch[1/5],step[600/600],Loss:0.0170
Epoch[2/5],step[100/600],Loss:0.0229
Epoch[2/5],step[200/600],Loss:0.0404
Epoch[2/5],step[300/600],Loss:0.0168
Epoch[2/5],step[400/600],Loss:0.0105
Epoch[2/5],step[500/600],Loss:0.0398
Epoch[2/5],step[600/600],Loss:0.0047
Epoch[3/5],step[100/600],Loss:0.0149
Epoch[3/5],step[200/600],Loss:0.0121
Epoch[3/5],step[300/600],Loss:0.0056
Epoch[3/5],step[400/600],Loss:0.1210
Epoch[3/5],step[500/600],Loss:0.0416
Epoch[3/5],step[600/600],Loss:0.1685
Epoch[4/5],step[100/600],Loss:0.0187
Epoch[4/5],step[200/600],Loss:0.0168
Epoch[4/5],step[300/600],Loss:0.0801
Epoch[4/5],step[400/600],Loss:0.0137
Epoch[4/5],step[500/600],Loss:0.0265
Epoch[4/5],step[600/600],Loss:0.0051
Epoch[5/5],step[100/600],Loss:0.0033
Epoch[5/5],step[200/600],Loss:0.0066
Epoch[5/5],step[300/600],Loss:0.0625
E

In [33]:
#eval mode (batch norm uses moving mean/variance instead of mini_batch mean/variance)
model.eval()
with torch.no_grad():
    correct=0
    total=0
    for images,labels in test_loader:
        images=images.to(device)
        labels=labels.to(device)
        outputs=model(images)
        _,predicted=torch.max(outputs.data,1)
        total +=labels.size(0)
        correct +=(predicted==labels).sum().item()
    print('Accuracy of the network on the 10000 test images:{}%'.format(100*correct/total))    

Accuracy of the network on the 10000 test images:98.69%
